# 한빛출판네트워크
- 제목, 저자, 역자, 출간일, 페이지, 정가, 판가, 마일리지

In [245]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import requests

In [246]:
url = 'https://www.hanbit.co.kr/store/books/new_book_list.html'
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')

In [247]:
lis = soup.select_one('.new_book_list_wrap > div.sub_book_list_area').select('li')
len(lis)
li = lis[0]

In [248]:
# 책 상세페이지 들어가기
href = li.select_one('a')['href']
sub_url = 'https://www.hanbit.co.kr' + href
sub_req = requests.get(sub_url)
sub_soup = BeautifulSoup(sub_req.text, 'html.parser')
sub_url
bi = sub_soup.select_one('#container > div.store_view_wrap > div.store_view_wrap_l > div.store_view_area > div.store_product_info_box')


In [249]:
# 제목
title =  bi.select_one('h3').get_text()
title

'하루 한 장 초등과학 365'

In [250]:
# 저자, 번역자, 출간일, 페이지
info = bi.select_one('ul').get_text().strip().split('\n')
info

['저자 : 치바 가즈요시 ',
 '번역 : 허영은 , 유우종 (감수) ',
 '출간 : 2021-12-14',
 '페이지 : 416 쪽',
 'ISBN : 9791190846295',
 '물류코드 :5329']

In [251]:
# 저자
author = info[0][5:].strip()
author

'치바 가즈요시'

In [252]:
# 역자
inter = info[1][5:].strip()
inter

'허영은 , 유우종 (감수)'

In [253]:
# 출간일
import re
# date = info[2][5:].replace('-','').strip()
date = re.sub('[^0-9]','',info[2][5:])
date

'20211214'

In [254]:
# 페이지
page = re.sub('[^0-9]','',info[3][6:])
page

'416'

In [255]:
# 정가, 판가, 마일리지
bi2 = sub_soup.select_one('#container > div.store_view_wrap > div.store_payment_area > fieldset > label.payment_box.curr').get_text().strip().split('\n')
bi2

['정가 : 19,800원', '판매가 : 17,820원(10% off)', '마일리지 : 990점 (5%)']

In [256]:
# 윗줄로 10페이지 적용하니 오류가 떠서 다시 찾은 가격 경로
price = re.sub('[^0-9]','',sub_soup.select_one('.payment_box.curr').select_one('p').get_text())
price

'19800'

In [257]:
# 정가
price = re.sub('[^0-9]','',bi2[0])
price

'19800'

In [267]:
# 다시 찾은 판매가
sell_price = re.sub('[^0-9]','',sub_soup.select('.payment_box.curr')[0].get_text().strip().split('\n')[1].split('(')[0])
sell_price

'17820'

In [224]:
# 판매가
sell_price = re.sub('[^0-9]','',bi2[1].split('(')[0])
sell_price

'17820'

In [269]:
# 다.찾.마
mileage = re.sub('[^0-9]','',sub_soup.select('.payment_box.curr')[0].get_text().strip().split('\n')[2].split('(')[0])
mileage

'400'

In [225]:
# 마일리지
mileage = re.sub('[^0-9]','',bi2[2].split('(')[0])
mileage

'990'

In [226]:
# 한 페이지에 적용
lines = []
for li in lis:
    href = li.select_one('a')['href']
    sub_url = 'https://www.hanbit.co.kr' + href
    sub_req = requests.get(sub_url)
    sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

    bi = sub_soup.select_one('#container > div.store_view_wrap > div.store_view_wrap_l > div.store_view_area > div.store_product_info_box')
    title =  bi.select_one('h3').get_text()     # 책제목

    info = bi.select_one('ul').get_text().strip().split('\n')   # 저자, 역자, 출간일, 페이지
    author = info[0][5:].strip()    # 저자
    

    if info[1][:2] == '번역':     # 역자가 포함된 경우
        inter = info[1][5:].strip() # 역자
        date = re.sub('[^0-9]','',info[2][5:]) # 출간일
        page = re.sub('[^0-9]','',info[3][6:]) # 페이지
    else:                   # 역자 정보가 없을 경우
        inter = '-'
        date = re.sub('[^0-9]','',info[1][5:])
        page = re.sub('[^0-9]','',info[2][6:])
    
    # 가격 정보
    bi2 = sub_soup.select_one('#container > div.store_view_wrap > div.store_payment_area > fieldset > label.payment_box.curr').get_text().strip().split('\n')
    
    price = re.sub('[^0-9]','',bi2[0]) # 정가
    sell_price = re.sub('[^0-9]','',bi2[1].split('(')[0]) # 판매가
    mileage = re.sub('[^0-9]','',bi2[2].split('(')[0]) # 마일리지

    lines.append([title, author, inter, date, page, price, sell_price, mileage])

In [227]:
df = pd.DataFrame(lines, columns=['제목', '저자', '역자', '출간일', '페이지', '정가', '판가', '마일리지'])
df

,제목,저자,역자,출간일,페이지,정가,판가,마일리지
0,하루 한 장 초등과학 365,치바 가즈요시,"허영은 , 유우종 (감수)",20211214,416,19800,17820,990
1,"다시, 배우다",폴 김,-,20211210,304,16000,14400,800
2,당당한 디자인 결정을 위한 9가지 방법,톰 그리버,"김민성 , 곽서희",20211210,376,22000,19800,1100
3,365 부모 말하기 연습 일력(스프링북),"박재연 , 공인영",-,20211201,384,18000,16200,900
4,공학도를 위한 수치해석(8판),"Steven C.Chapra , Raymond P. Canale","나양 , 이승배 , 임윤철",20211130,600,36000,36000,1080
5,엑셀 매크로&VBA 업무 공략집,최준선,-,20211130,328,22000,19800,1100
6,데이터 과학자 되는 법,"에밀리 로빈슨 , 재클린 놀리스",이창화,20211126,372,24000,21600,1200
7,돈 되는 기획,김도균,-,20211126,240,14500,13050,725
8,빠르게 찾아 바로 적용하는 엑셀 에센스 사전 100,한은숙,-,20211125,300,18000,16200,900
9,초등 어휘력이 공부력이다,박명선,-,20211125,412,18000,16200,900


In [277]:
# 10페이지까지 적용...하니까 한페이지 적용의 # 가격 정보부터 에러가 떠서 가격, 판매가, 마일리지 경로 다시 찾아서 해결

lines = []
for page in range(1, 11):
    url = f'https://www.hanbit.co.kr/store/books/new_book_list.html?page={page}&brand=&cate1=&cate2=&searchKey=&keyWord='
    req = requests.get(url)
    soup = BeautifulSoup(req.text, 'html.parser')
    lis = soup.select_one('.new_book_list_wrap > div.sub_book_list_area').select('li')

    for li in lis:
        href = li.select_one('a')['href']
        sub_url = 'https://www.hanbit.co.kr' + href
        sub_req = requests.get(sub_url)
        sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

        bi = sub_soup.select_one('#container > div.store_view_wrap > div.store_view_wrap_l > div.store_view_area > div.store_product_info_box')
        title =  bi.select_one('h3').get_text()     # 책제목

        info = bi.select_one('ul').get_text().strip().split('\n')   # 저자, 역자, 출간일, 페이지
        author = info[0][5:].strip()    # 저자
        

        if info[1][:2] == '번역':     # 역자가 포함된 경우
            inter = info[1][5:].strip() # 역자
            date = re.sub('[^0-9]','',info[2][5:]) # 출간일
            page = re.sub('[^0-9]','',info[3][6:]) # 페이지
        else:                   # 역자 정보가 없을 경우
            inter = '-'
            date = re.sub('[^0-9]','',info[1][5:])
            page = re.sub('[^0-9]','',info[2][6:])
        
        price = re.sub('[^0-9]','',sub_soup.select_one('.payment_box.curr').select_one('p').get_text())
        sell_price = re.sub('[^0-9]','',sub_soup.select('.payment_box.curr')[0].get_text().strip().split('\n')[1].split('(')[0])
        mileage = re.sub('[^0-9]','',sub_soup.select('.payment_box.curr')[0].get_text().strip().split('\n')[2].split('(')[0])
        lines.append([title, author, inter, date, page, price, sell_price, mileage])

In [278]:
# 최종!!
df = pd.DataFrame(lines, columns=['제목', '저자', '역자', '출간일', '페이지', '정가', '판가', '마일리지'])
df

,제목,저자,역자,출간일,페이지,정가,판가,마일리지
0,하루 한 장 초등과학 365,치바 가즈요시,"허영은 , 유우종 (감수)",20211214,416,19800,17820,990
1,"다시, 배우다",폴 김,-,20211210,304,16000,14400,800
2,당당한 디자인 결정을 위한 9가지 방법,톰 그리버,"김민성 , 곽서희",20211210,376,22000,19800,1100
3,365 부모 말하기 연습 일력(스프링북),"박재연 , 공인영",-,20211201,384,18000,16200,900
4,공학도를 위한 수치해석(8판),"Steven C.Chapra , Raymond P. Canale","나양 , 이승배 , 임윤철",20211130,600,36000,36000,1080
...,...,...,...,...,...,...,...,...
195,"IT CookBook, C 언어 일취월장(2판)",신윤환,-,20210224,800,28000,28000,840
196,소비자행동,김학윤,-,20210223,660,35000,35000,1050
197,재미있고 빠른 어휘력 동화 – 두근두근 이 마음은 뭘까,"김세실 , 김도윤(그림)",-,20210220,104,12000,10800,600
198,재미있고 빠른 어휘력 동화 – 키득키득 재밌는 의성어 의태어,"김예실 , 김희선(그림)",-,20210220,104,12000,10800,600


### 이하는 에러나는 곳 확인했던 작업

In [272]:
# 결과 1페이지부터 에러 떠서 가격 정보 경로를 다시 찾음. copy selector를 맹신한 게 문제였을까

lines = []
url = 'https://www.hanbit.co.kr/store/books/new_book_list.html?page=1&brand=&cate1=&cate2=&searchKey=&keyWord='
req = requests.get(url)
soup = BeautifulSoup(req.text, 'html.parser')
lis = soup.select_one('.new_book_list_wrap > div.sub_book_list_area').select('li')

for li in lis:
    href = li.select_one('a')['href']
    sub_url = 'https://www.hanbit.co.kr' + href
    sub_req = requests.get(sub_url)
    sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

    bi = sub_soup.select_one('#container > div.store_view_wrap > div.store_view_wrap_l > div.store_view_area > div.store_product_info_box')
    title =  bi.select_one('h3').get_text()     # 책제목

    info = bi.select_one('ul').get_text().strip().split('\n')   # 저자, 역자, 출간일, 페이지
    author = info[0][5:].strip()    # 저자
    

    if info[1][:2] == '번역':     # 역자가 포함된 경우
        inter = info[1][5:].strip() # 역자
        date = re.sub('[^0-9]','',info[2][5:]) # 출간일
        page = re.sub('[^0-9]','',info[3][6:]) # 페이지
    else:                   # 역자 정보가 없을 경우
        inter = '-'
        date = re.sub('[^0-9]','',info[1][5:])
        page = re.sub('[^0-9]','',info[2][6:])
    
    # 가격 정보
    # bi2 = sub_soup.select_one('#container > div.store_view_wrap > div.store_payment_area > fieldset > label.payment_box.curr').get_text().strip().split('\n')
    price = re.sub('[^0-9]','',sub_soup.select_one('.payment_box.curr').select_one('p').get_text())
    sell_price = re.sub('[^0-9]','',sub_soup.select('.payment_box.curr')[0].get_text().strip().split('\n')[1].split('(')[0])
    mileage = re.sub('[^0-9]','',sub_soup.select('.payment_box.curr')[0].get_text().strip().split('\n')[2].split('(')[0])
    # price = re.sub('[^0-9]','',bi2.get_text().strip().split('\n')[0]) # 정가
    # sell_price = re.sub('[^0-9]','',bi2.get_text().strip().split('\n')[1].split('(')[0]) # 판매가
    # mileage = re.sub('[^0-9]','',bi2.get_text().strip().split('\n')[2].split('(')[0]) # 마일리지
    lines.append([title, author, inter, date, page, price, sell_price, mileage])
    # lines.append([title, author, inter, date, page, price, sell_price, mileage])

In [273]:
df = pd.DataFrame(lines, columns=['제목', '저자', '역자', '출간일', '페이지', '정가', '판가', '마일리지'])
df

,제목,저자,역자,출간일,페이지,정가,판가,마일리지
0,하루 한 장 초등과학 365,치바 가즈요시,"허영은 , 유우종 (감수)",20211214,416,19800,17820,990
1,"다시, 배우다",폴 김,-,20211210,304,16000,14400,800
2,당당한 디자인 결정을 위한 9가지 방법,톰 그리버,"김민성 , 곽서희",20211210,376,22000,19800,1100
3,365 부모 말하기 연습 일력(스프링북),"박재연 , 공인영",-,20211201,384,18000,16200,900
4,공학도를 위한 수치해석(8판),"Steven C.Chapra , Raymond P. Canale","나양 , 이승배 , 임윤철",20211130,600,36000,36000,1080
5,엑셀 매크로&VBA 업무 공략집,최준선,-,20211130,328,22000,19800,1100
6,데이터 과학자 되는 법,"에밀리 로빈슨 , 재클린 놀리스",이창화,20211126,372,24000,21600,1200
7,돈 되는 기획,김도균,-,20211126,240,14500,13050,725
8,빠르게 찾아 바로 적용하는 엑셀 에센스 사전 100,한은숙,-,20211125,300,18000,16200,900
9,초등 어휘력이 공부력이다,박명선,-,20211125,412,18000,16200,900
